# Live Calibration Session

**Purpose**: Fast iteration cycle for live parameter calibration

**Workflow**: Load → Modify → Run → Visualize → Repeat

**This is throwaway code** - for SCRI session only

---

## Setup: Import Engine

In [ ]:
import sys
sys.path.insert(0, '../..')

from seleensim.entities import Site, Trial, PatientFlow
from seleensim.distributions import Triangular, Gamma, Bernoulli
from seleensim.simulation import SimulationEngine
from seleensim.constraints import (
    BudgetThrottlingConstraint,
    ResourceCapacityConstraint,
    LinearResponseCurve,
    LinearCapacityDegradation,
    NoCapacityDegradation
)

import json
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

print("✓ Engine imported (read-only)")

---

## Cell 1: Load Baseline Configuration

**What this does**: Creates baseline trial and constraint objects using engine constructors

**Instructions**: Run once at start of session

**These are actual engine parameters** - no invented concepts

In [ ]:
# Build baseline trial (engine objects only)
site = Site(
    site_id="SITE_001",
    activation_time=Triangular(low=30, mode=45, high=90),
    enrollment_rate=Gamma(shape=2, scale=1.5),
    dropout_rate=Bernoulli(p=0.15)
)

flow = PatientFlow(
    flow_id="STANDARD_FLOW",
    states={"enrolled", "completed"},
    initial_state="enrolled",
    terminal_states={"completed"},
    transition_times={
        ("enrolled", "completed"): Triangular(low=90, mode=180, high=365)
    }
)

trial = Trial(
    trial_id="LIVE_CALIBRATION",
    target_enrollment=200,
    sites=[site],
    patient_flow=flow
)

# Baseline constraint parameters (engine defaults)
# These will be modified in Cell 2
baseline_budget_per_day = 50000
baseline_min_speed_ratio = 0.5
baseline_threshold = 0.8
baseline_max_multiplier = 2.0
baseline_max_utilization = 1.5

# Simulation parameters
master_seed = 42
num_runs = 100

print("✓ Baseline configuration loaded")
print(f"\nTrial: {trial.target_enrollment} patients")
print(f"Budget: ${baseline_budget_per_day:,}/day, {1/baseline_min_speed_ratio:.1f}x max slowdown")
print(f"Capacity: {baseline_threshold*100:.0f}% threshold, {baseline_max_multiplier:.1f}x max slowdown")
print(f"Simulation: {num_runs} runs")

---

## Cell 2: Modify Engine Parameters

**What this does**: Change constraint constructor parameters

**Instructions**: 
1. SCRI suggests new values
2. Update the parameters below
3. Re-run this cell
4. Proceed to Cell 3

**These map 1:1 to engine inputs** - no intermediate abstractions

In [ ]:
# 👉 SCRI: CHANGE THESE ENGINE PARAMETERS

# BudgetThrottlingConstraint parameters
budget_per_day = 50000      # Constructor: BudgetThrottlingConstraint(budget_per_day=...)
min_speed_ratio = 0.5       # Constructor: LinearResponseCurve(min_speed_ratio=...)

# ResourceCapacityConstraint parameters
# Option A: Use LinearCapacityDegradation
threshold = 0.8             # Constructor: LinearCapacityDegradation(threshold=...)
max_multiplier = 2.0        # Constructor: LinearCapacityDegradation(max_multiplier=...)
max_utilization = 1.5       # Constructor: LinearCapacityDegradation(max_utilization=...)

# Option B: To switch to NoCapacityDegradation, uncomment line below and skip to Cell 3
# capacity_response = NoCapacityDegradation()
# Then comment out the LinearCapacityDegradation line in Cell 3

print("✓ Engine parameters updated:")
print(f"\n  BudgetThrottlingConstraint:")
print(f"    budget_per_day = ${budget_per_day:,}")
print(f"  LinearResponseCurve:")
print(f"    min_speed_ratio = {min_speed_ratio} ({1/min_speed_ratio:.1f}x max slowdown)")
print(f"\n  LinearCapacityDegradation:")
print(f"    threshold = {threshold}")
print(f"    max_multiplier = {max_multiplier}")
print(f"    max_utilization = {max_utilization}")
print(f"\n  → Ready for Cell 3")

---

## Cell 3: Run Simulation

**What this does**: Constructs engine objects with parameters from Cell 2, runs simulation

**Instructions**: Run this after Cell 2

**Expected runtime**: ~5-10 seconds for 100 runs

In [ ]:
# Construct engine objects (exact 1:1 mapping)
budget_constraint = BudgetThrottlingConstraint(
    budget_per_day=budget_per_day,
    response_curve=LinearResponseCurve(min_speed_ratio=min_speed_ratio)
)

capacity_constraint = ResourceCapacityConstraint(
    resource_id="CRA",
    capacity_response=LinearCapacityDegradation(
        threshold=threshold,
        max_multiplier=max_multiplier,
        max_utilization=max_utilization
    )
    # Or use: capacity_response=NoCapacityDegradation()
)

# Run simulation
print("Running simulation...")
engine = SimulationEngine(master_seed=master_seed, constraints=[budget_constraint, capacity_constraint])
results = engine.run(trial, num_runs=num_runs)

print("\n✓ Simulation complete")
print(f"\nResults ({num_runs} runs):")
print(f"  P10 completion: {results.completion_time_p10:.1f} days")
print(f"  P50 completion: {results.completion_time_p50:.1f} days")
print(f"  P90 completion: {results.completion_time_p90:.1f} days")
print(f"  Mean events rescheduled: {results.mean_events_rescheduled:.1f}")
print(f"\n  → Proceed to Cell 4")

---

## Cell 4: Visualize Outputs

**What this does**: Simple plots of simulation results

**Instructions**: 
1. Review with SCRI
2. Ask: "Does this match your experience?"
3. If not, return to Cell 2, adjust parameters, re-run
4. Repeat until SCRI says "That feels right"

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Histogram
axes[0].hist(results.completion_times, bins=30, alpha=0.7, edgecolor='black', color='steelblue')
axes[0].axvline(results.completion_time_p10, color='green', linestyle='--', linewidth=2, label='P10')
axes[0].axvline(results.completion_time_p50, color='red', linestyle='--', linewidth=2, label='P50')
axes[0].axvline(results.completion_time_p90, color='orange', linestyle='--', linewidth=2, label='P90')
axes[0].set_xlabel('Completion Time (days)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Trial Completion Time Distribution', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(alpha=0.3)

# Plot 2: Box plot
bp = axes[1].boxplot([results.completion_times], widths=0.5, patch_artist=True)
bp['boxes'][0].set_facecolor('steelblue')
bp['boxes'][0].set_alpha(0.7)
axes[1].set_ylabel('Completion Time (days)', fontsize=12)
axes[1].set_title('Distribution Summary', fontsize=14, fontweight='bold')
axes[1].set_xticklabels(['Trial'])
axes[1].grid(alpha=0.3, axis='y')

textstr = f"P10: {results.completion_time_p10:.1f} days\n"
textstr += f"P50: {results.completion_time_p50:.1f} days\n"
textstr += f"P90: {results.completion_time_p90:.1f} days\n"
textstr += f"Events rescheduled: {results.mean_events_rescheduled:.1f}"
axes[1].text(0.02, 0.98, textstr, transform=axes[1].transAxes, fontsize=10,
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

# Summary
print("\n" + "="*50)
print("SUMMARY")
print("="*50)
print(f"\nCompletion Time:")
print(f"  P10:  {results.completion_time_p10:.1f} days")
print(f"  P50:  {results.completion_time_p50:.1f} days")
print(f"  P90:  {results.completion_time_p90:.1f} days")
print(f"  Range (P10-P90): {results.completion_time_p90 - results.completion_time_p10:.1f} days")
print(f"\nConstraint Impact:")
print(f"  Events rescheduled: {results.mean_events_rescheduled:.1f}")
print("\n" + "="*50)

---

## Questions for SCRI

After viewing results:

1. **Does this match your experience?**
   - Is P50 reasonable?
   - Is the range realistic?

2. **What would you change?**
   - More budget pressure? (lower `min_speed_ratio`)
   - Capacity degrades earlier? (lower `threshold`)
   - Worse degradation? (higher `max_multiplier`)

3. **Direction test:**
   - If we increase `max_multiplier`, P90 should increase (slower) - make sense?
   - If we increase `budget_per_day`, P90 should decrease (faster) - make sense?

**If SCRI wants changes:** Go to Cell 2 → Update parameters → Re-run Cell 2, 3, 4

---

## Save Calibrated Parameters

**When**: After SCRI says "That feels right"

**What this does**: Saves final engine parameter values to JSON

In [ ]:
# Capture final engine parameters (no invented structure)
calibrated = {
    "session_date": datetime.now().isoformat(),
    "engine_parameters": {
        "BudgetThrottlingConstraint": {
            "budget_per_day": budget_per_day
        },
        "LinearResponseCurve": {
            "min_speed_ratio": min_speed_ratio
        },
        "ResourceCapacityConstraint": {
            "resource_id": "CRA"
        },
        "LinearCapacityDegradation": {
            "threshold": threshold,
            "max_multiplier": max_multiplier,
            "max_utilization": max_utilization
        },
        "SimulationEngine": {
            "master_seed": master_seed
        },
        "engine.run": {
            "num_runs": num_runs
        }
    },
    "results": {
        "p10_days": float(results.completion_time_p10),
        "p50_days": float(results.completion_time_p50),
        "p90_days": float(results.completion_time_p90),
        "mean_events_rescheduled": float(results.mean_events_rescheduled)
    }
}

filename = f"calibrated_params_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename, 'w') as f:
    json.dump(calibrated, f, indent=2)

print(f"✓ Saved to: {filename}")
print("\nFinal engine parameters:")
print(json.dumps(calibrated['engine_parameters'], indent=2))
print(f"\nResults: P50={calibrated['results']['p50_days']:.1f} days, P90={calibrated['results']['p90_days']:.1f} days")

---

## Session Notes

### Iteration History
| # | budget_per_day | min_speed_ratio | threshold | max_multiplier | P50 | P90 | Feedback |
|---|----------------|-----------------|-----------|----------------|-----|-----|----------|
| 1 |                |                 |           |                |     |     |          |
| 2 |                |                 |           |                |     |     |          |
| 3 |                |                 |           |                |     |     | ✓ Approved |

### Insights
- What matched SCRI intuition?
- What was surprising?
- Most sensitive parameters?

### Architecture Validation
- Can SCRI express reality using these engine parameters? Yes / No
- Gaps identified:
- Next steps:

---